In [1]:
!pip install opencv-python==4.7.0.72


In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import drive
from PIL import Image

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def visualize(input, faces, thickness=2):
    if faces[1] is not None:
        for idx, face in enumerate(faces[1]):
            coords = face[:-1].astype(np.int32)
            cv2.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), thickness)
            #cv2.circle(input, (coords[4], coords[5]), 2, (255, 0, 0), thickness)
            #cv2.circle(input, (coords[6], coords[7]), 2, (0, 0, 255), thickness)
            #cv2.circle(input, (coords[8], coords[9]), 2, (0, 255, 0), thickness)
            #cv2.circle(input, (coords[10], coords[11]), 2, (255, 0, 255), thickness)
            #cv2.circle(input, (coords[12], coords[13]), 2, (0, 255, 255), thickness)

In [5]:
detector = cv2.FaceDetectorYN.create(
    "/content/drive/MyDrive/Ai_Lab/cods/face/face_detection_yunet_2022mar.onnx",
    "",
    (320, 320),
    0.8,
    0.3,
    5000
)
recognizer = cv2.FaceRecognizerSF.create(
    "/content/drive/MyDrive/Ai_Lab/cods/face/face_recognition_sface_2021dec.onnx", ""
)

## Read Images

In [6]:
# Load known images and extract features
known_faces = {
    "yazdaniyan": [
        "/content/drive/MyDrive/Ai_Lab/data/Yazdanian/simple/51101028.JPG",
        "/content/drive/MyDrive/Ai_Lab/data/Yazdanian/simple/51101028.JPG",
        "/content/drive/MyDrive/Ai_Lab/data/Yazdanian/simple/51101028.JPG"
    ],
    "akhavan": [
        "/content/drive/MyDrive/Ai_Lab/data/akhavan/simple/51251038.png",
        "/content/drive/MyDrive/Ai_Lab/data/akhavan/simple/51251038.png",
        "/content/drive/MyDrive/Ai_Lab/data/akhavan/simple/51251038.png"
    ]
}


## Train part

In [7]:
known_features = {}
for name, image_paths in known_faces.items():
    features = []
    for image_path in image_paths:
        image = cv2.imread(image_path)
        imgWidth = int(image.shape[1])
        imgHeight = int(image.shape[0])
        detector.setInputSize((imgWidth, imgHeight))
        faces = detector.detect(image)
        assert faces[1] is not None, f'Cannot find a face in {image_path}'

        # Align and extract features
        face_align = recognizer.alignCrop(image, faces[1][0])
        face_feature = recognizer.feature(face_align)
        features.append(face_feature)

    # Average the features for this person
    avg_feature = np.mean(features, axis=0)
    known_features[name] = avg_feature

##Test part

In [8]:
# Load test image
test_image1 = cv2.imread(r"/content/drive/MyDrive/Ai_Lab/data/akhavan/simple/51251001.jpg")
test_image = test_image1.copy()
test_imgWidth = int(test_image.shape[1])
test_imgHeight = int(test_image.shape[0])
detector.setInputSize((test_imgWidth, test_imgHeight))
test_faces = detector.detect(test_image)
assert test_faces[1] is not None, 'Cannot find a face in the test image'

# Align and extract features of the test face
test_face_align = recognizer.alignCrop(test_image, test_faces[1][0])
test_face_feature = recognizer.feature(test_face_align)

## calculate similarity(l2_score and cosion)

In [9]:
# Compare test face with known faces
best_match_name = "Unknown"
best_match_score = float('inf')  # For L2 distance (lower is better)
best_cosine_score = -1  # For cosine similarity (higher is better)

for name, known_feature in known_features.items():
    # Calculate L2 distance
    l2_score = recognizer.match(test_face_feature, known_feature, cv2.FaceRecognizerSF_FR_NORM_L2)

    # Calculate cosine similarity
    cosine_score = recognizer.match(test_face_feature, known_feature, cv2.FaceRecognizerSF_FR_COSINE)

    # Update best match
    if l2_score < best_match_score:
        best_match_score = l2_score
        best_match_name = name
        best_cosine_score = cosine_score

# Define similarity thresholds
l2_similarity_threshold = 1.128
cosine_similarity_threshold = 0.363

##Result

In [10]:
# Determine if the match is valid
if best_match_score <= l2_similarity_threshold or best_cosine_score >= cosine_similarity_threshold:
    print(f"The test image is recognized as: {best_match_name}")
    print(f"NormL2 Distance: {best_match_score}, Cosine Similarity: {best_cosine_score}")
else:
    print("The test image does not match any known person.")

The test image is recognized as: akhavan
NormL2 Distance: 0.7668440186458264, Cosine Similarity: 0.705975060599485
